In [6]:
import tensorflow as tf
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import torch
import zipfile

from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from PIL import Image
from google.colab import files

# Cargar datos

In [ ]:
uploaded = files.upload()

In [ ]:
directorio_actual = os.getcwd()
print("Estás trabajando en:", directorio_actual)

In [ ]:
with zipfile.ZipFile("Dogs_vs_cats.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
print(os.listdir("."))

In [ ]:
# Conteo de archivos en cada carpeta
num_gatos = len(os.listdir(carpeta_gatos))
num_perros = len(os.listdir(carpeta_perros))

print(f"Imágenes de gatos: {num_gatos}")
print(f"Imágenes de perros: {num_perros}")

In [ ]:
# Indicación de las rutas para datos de entrenamiento y para test
train_path = '/content/dogs_vs_cats/train'
test_path = '/content/dogs_vs_cats/test'

In [ ]:
# Reescalamos los valores de píxeles a [0, 1]
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
trdata = datagen.flow_from_directory(
    train_path,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical'
)

tsdata = datagen.flow_from_directory(
    test_path,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical'
)

# Visualizar la información de entrada

In [ ]:
# Usamos una tanda para obtener las imágenes
batch_images, batch_labels = next(trdata)
class_names = trdata.class_indices
clase_idx_inv = {v: k for k, v in class_names.items()}

In [ ]:
def mostrar_clase(imagenes, etiquetas, clase_id, nombre):
    clase_imgs = imagenes[np.argmax(etiquetas, axis=1) == clase_id][:9]

    plt.figure(figsize=(6,6))
    for i in range(len(clase_imgs)):
        plt.subplot(3, 3, i+1)
        plt.imshow(clase_imgs[i])
        plt.axis("off")
    plt.suptitle(f"{nombre}", fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
mostrar_clase(batch_images, batch_labels, class_names['cats'], 'Gatos')
mostrar_clase(batch_images, batch_labels, class_names['dogs'], 'Perros')

# Construcción del modelo CNN

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(200, 200, 3)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Optimizar modelo

In [ ]:
# Guardar solo el mejor modelo según validación
checkpoint = ModelCheckpoint(
    'mejor_modelo.h5', monitor='val_accuracy',
    save_best_only=True, verbose=1
)
# Parar si no mejora tras 3 épocas seguidas
earlystop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Entrenamiento
history = model.fit(
    trdata,
    validation_data=tsdata,
    epochs=20,
    callbacks=[checkpoint, earlystop],
    verbose=1
)